In [1]:
from PPORewardRecall import RewardCallback
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
import stable_baselines3
import gymnasium as gym

from FrozenLakeRewardWrapper import FrozenLakeRewardWrapper
from PPOAgent import PPOAgent
from stable_baselines3 import PPO

# 创建 FrozenLake 环境 (Q-Learning)
map = generate_random_map(size=16)
training_env1 = gym.make('FrozenLake-v1', desc=map, is_slippery = False)#map_name='4x4', is_slippery=True)#
wrapped_training_env = FrozenLakeRewardWrapper(training_env1)

eval_env = gym.make('FrozenLake-v1', desc=map, is_slippery = False)#map_name='4x4', is_slippery=True）#
wrapped_eval_env = FrozenLakeRewardWrapper(eval_env)

# agent = QLearningAgent(wrapped_training_env)
model = PPO(
    policy="MlpPolicy",            # 使用多层感知机策略
    env=wrapped_training_env,      # 传入包装后的训练环境
    verbose=0,                     # 显示训练过程的详细信息
    learning_rate=5e-4,            # 设置学习率
    clip_range=0.3,                # 设置剪切范围（epsilon_clip）
    n_epochs=3,                    # 训练周期数
    batch_size=128,                 # 批次大小
    gamma=0.99,                    # 折扣因子
    gae_lambda=0.95,               # GAE 的 lambda 参数
    device='cpu',
    policy_kwargs=dict(net_arch=[64, 64]),
    ent_coef= 0.02
)
reward_callback = RewardCallback(
    eval_env=wrapped_eval_env,
    eval_freq=5000,
    n_eval_episodes=100,
    verbose=1
)

print("Start training...")
#agent.train(episodes=3000)
model.learn(
    total_timesteps=500000,
    callback=reward_callback
)
wrapped_training_env.close()

Start training...


D:\Anaconda3\envs\final\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Step 5000	平均奖励: -19.30 +/- 0.00
Step 10000	平均奖励: -6.00 +/- 0.00
Step 15000	平均奖励: -3.50 +/- 0.00
Step 20000	平均奖励: -1.80 +/- 0.00
Step 25000	平均奖励: 34.50 +/- 0.00
Step 30000	平均奖励: 34.50 +/- 0.00
Step 35000	平均奖励: 34.50 +/- 0.00
Step 40000	平均奖励: 34.50 +/- 0.00
Step 45000	平均奖励: 34.50 +/- 0.00
Step 50000	平均奖励: 34.50 +/- 0.00
Step 55000	平均奖励: 34.50 +/- 0.00
Step 60000	平均奖励: 34.50 +/- 0.00
Step 65000	平均奖励: 34.50 +/- 0.00
Step 70000	平均奖励: 34.50 +/- 0.00
Step 75000	平均奖励: 34.50 +/- 0.00
Step 80000	平均奖励: 34.50 +/- 0.00
Step 85000	平均奖励: 34.50 +/- 0.00
Step 90000	平均奖励: 34.50 +/- 0.00
Step 95000	平均奖励: 34.50 +/- 0.00
Step 100000	平均奖励: 34.50 +/- 0.00
Step 105000	平均奖励: 34.50 +/- 0.00
Step 110000	平均奖励: 34.50 +/- 0.00
Step 115000	平均奖励: 34.50 +/- 0.00
Step 120000	平均奖励: 34.50 +/- 0.00
Step 125000	平均奖励: 34.50 +/- 0.00
Step 130000	平均奖励: 34.50 +/- 0.00
Step 135000	平均奖励: 34.50 +/- 0.00
Step 140000	平均奖励: 34.50 +/- 0.00
Step 145000	平均奖励: 34.50 +/- 0.00
Step 150000	平均奖励: 34.50 +/- 0.00
Step 155000	平均奖励: 34.50 +/- 0

In [2]:
from stable_baselines3.common.evaluation import evaluate_policy

'''
agent.env = wrapped_eval_env
print("Start evaluation...")
agent.evaluate(episodes=100)
eval_env.close()
'''
print("Start evaluation...")
mean_reward, std_reward = evaluate_policy(
    model,
    wrapped_eval_env,
    n_eval_episodes=100,
    deterministic=True,  # 使用确定性策略进行评估
    render=False
)
print(f"Final Evaluation\t平均奖励: {mean_reward:.2f} +/- {std_reward:.2f}")


Start evaluation...
Final Evaluation	平均奖励: 34.50 +/- 0.00


In [3]:
final_env = gym.make('FrozenLake-v1', is_slippery=False, render_mode = 'human', desc = map)#,map_name='4x4')#
wrapped_final_env = FrozenLakeRewardWrapper(final_env)

#agent.env = wrapped_final_env
#model.set_env(wrapped_final_env)

# 重置环境，获取初始状态
state, info = wrapped_final_env.reset()

done = False  # 游戏是否结束
step_count = 0  # 计步

while not done:
    wrapped_final_env.render()  # 渲染环境
    #action = agent.choose_best_action(state)  # 选择动作
    action, _ = model.predict(state, deterministic=True)
    state, reward, terminated, truncated, _ = wrapped_final_env.step(action[()])  # 执行动作
    done = terminated or truncated
    print(f"Step {step_count}: Action={action}, State={state}, Reward={reward}, Done={done}")
    step_count += 1

# 游戏结束后关闭环境
wrapped_final_env.close()


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


Step 0: Action=1, State=16, Reward=0.5, Done=False
Step 1: Action=1, State=32, Reward=0.5, Done=False
Step 2: Action=2, State=33, Reward=0.5, Done=False
Step 3: Action=1, State=49, Reward=0.5, Done=False
Step 4: Action=2, State=50, Reward=0.5, Done=False
Step 5: Action=2, State=51, Reward=0.5, Done=False
Step 6: Action=2, State=52, Reward=0.5, Done=False
Step 7: Action=2, State=53, Reward=0.5, Done=False
Step 8: Action=2, State=54, Reward=0.5, Done=False
Step 9: Action=1, State=70, Reward=0.5, Done=False
Step 10: Action=1, State=86, Reward=0.5, Done=False
Step 11: Action=1, State=102, Reward=0.5, Done=False
Step 12: Action=1, State=118, Reward=0.5, Done=False
Step 13: Action=1, State=134, Reward=0.5, Done=False
Step 14: Action=2, State=135, Reward=0.5, Done=False
Step 15: Action=1, State=151, Reward=0.5, Done=False
Step 16: Action=2, State=152, Reward=0.5, Done=False
Step 17: Action=2, State=153, Reward=0.5, Done=False
Step 18: Action=1, State=169, Reward=0.5, Done=False
Step 19: Actio